# Territorial Anomalies Map

This notebook creates a visualization of non-standard territorial entities in Spain:

- **Shared Land (53xxx)**: Mancomunidades, Parzonerías, Facerías, Comunidades
- **Special Territories (54xxx)**: Plazas de Soberanía, Gibraltar, Condominio
- **Historical Mergers**: Municipalities that merged in recent years

**Output**: High-resolution map (DPI 369) showing these entities in context.

In [ ]:
# Standard library
from pathlib import Path
import warnings

# Third-party libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D

warnings.filterwarnings('ignore')

In [ ]:
# Configure paths
DATA_DIR = Path("/workspaces/rural-migration-land-use-spain/data")
SPATIAL_DIR = DATA_DIR / "spatial" / "raw"
OUTPUT_DIR = DATA_DIR / "maps"

# Create output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# CRS for all layers
TARGET_CRS = "EPSG:4258"

print(f"Target CRS: {TARGET_CRS} (ETRS89)")

## 1. Load spatial data

In [ ]:
# Load municipalities (Peninsula + Baleares)
print("Loading municipalities (Peninsula + Baleares)...")
mun_peninsula_shp = SPATIAL_DIR / "recintos_municipales_inspire_peninbal_etrs89.shp"
gdf_mun_peninsula = gpd.read_file(mun_peninsula_shp)
gdf_mun_peninsula = gdf_mun_peninsula.to_crs(TARGET_CRS)
print(f"   Loaded {len(gdf_mun_peninsula)} municipalities")

In [ ]:
# Load municipalities (Canarias)
print("Loading municipalities (Canarias)...")
mun_canarias_shp = SPATIAL_DIR / "recintos_municipales_inspire_canarias_regcan95.shp"
gdf_mun_canarias = gpd.read_file(mun_canarias_shp)
gdf_mun_canarias = gdf_mun_canarias.to_crs(TARGET_CRS)
print(f"   Loaded {len(gdf_mun_canarias)} municipalities")

In [ ]:
# Merge municipalities
print("Merging municipalities...")
gdf_mun = pd.concat([gdf_mun_peninsula, gdf_mun_canarias], ignore_index=True)
print(f"   Total: {len(gdf_mun)} municipalities")

In [ ]:
# Extract administrative codes from NATCODE
print("Extracting administrative codes...")
gdf_mun['NATCODE_str'] = gdf_mun['NATCODE'].astype(str).str.replace('ES', '', regex=False)
gdf_mun['Mun_Code'] = gdf_mun['NATCODE_str'].str[4:9]
gdf_mun['Prov_Code'] = gdf_mun['NATCODE_str'].str[2:4]
gdf_mun['CCAA_Code'] = gdf_mun['NATCODE_str'].str[0:2]
gdf_mun['Mun_Name'] = gdf_mun['NAMEUNIT']
print("   ✓ Codes extracted")

In [ ]:
# Load provinces
print("Loading provinces...")
prov_peninsula_shp = SPATIAL_DIR / "recintos_provinciales_inspire_peninbal_etrs89.shp"
prov_canarias_shp = SPATIAL_DIR / "recintos_provinciales_inspire_canarias_regcan95.shp"

gdf_prov_peninsula = gpd.read_file(prov_peninsula_shp).to_crs(TARGET_CRS)
gdf_prov_canarias = gpd.read_file(prov_canarias_shp).to_crs(TARGET_CRS)
gdf_prov = pd.concat([gdf_prov_peninsula, gdf_prov_canarias], ignore_index=True)
print(f"   Loaded {len(gdf_prov)} provinces")

In [ ]:
# Load CCAA
print("Loading autonomous communities...")
ccaa_peninsula_shp = SPATIAL_DIR / "recintos_autonomicas_inspire_peninbal_etrs89.shp"
ccaa_canarias_shp = SPATIAL_DIR / "recintos_autonomicas_inspire_canarias_regcan95.shp"

gdf_ccaa_peninsula = gpd.read_file(ccaa_peninsula_shp).to_crs(TARGET_CRS)
gdf_ccaa_canarias = gpd.read_file(ccaa_canarias_shp).to_crs(TARGET_CRS)
gdf_ccaa = pd.concat([gdf_ccaa_peninsula, gdf_ccaa_canarias], ignore_index=True)
print(f"   Loaded {len(gdf_ccaa)} autonomous communities")

## 2. Identify territorial anomalies

In [ ]:
# Extract non-municipality territories (codes >= 53000)
print("Identifying territorial anomalies...")
gdf_mun['Mun_Code_int'] = gdf_mun['Mun_Code'].astype(int)

# Shared land (53xxx)
shared_land = gdf_mun[
    (gdf_mun['Mun_Code_int'] >= 53000) &
    (gdf_mun['Mun_Code_int'] < 54000)
].copy()

# Special territories (54xxx)
special_territories = gdf_mun[
    gdf_mun['Mun_Code_int'] >= 54000
].copy()

print(f"   Shared land (53xxx): {len(shared_land)}")
print(f"   Special territories (54xxx): {len(special_territories)}")

## 3. Define historical mergers

In [ ]:
# Define historical mergers: new_code ← old_codes (ALL AS STRINGS)
HISTORICAL_MERGERS = {
    '15902': {
        'new_name': 'Oza-Cesuras',
        'merge_year': 2013,
        'old_codes': {
            '15026': 'Cesuras',
            '15063': 'Oza dos Ríos'
        }
    },
    '36902': {
        'new_name': 'Cerdedo-Cotobade',
        'merge_year': 2016,
        'old_codes': {
            '36011': 'Cerdedo',
            '36012': 'Cotobade'
        }
    }
}

# Extract historical merger geometries
historical_merger_codes = list(HISTORICAL_MERGERS.keys())
historical_mergers = gdf_mun[gdf_mun['Mun_Code'].isin(historical_merger_codes)].copy()

print(f"Historical mergers identified: {len(historical_mergers)}")
for code, info in HISTORICAL_MERGERS.items():
    print(f"   {code} - {info['new_name']} ({info['merge_year']})")

## 4. Create map visualization

In [ ]:
# ==============================================================================
# MAP: Territorial Anomalies (Peninsula focus + Canary Islands inset)
# ==============================================================================

fig, ax = plt.subplots(1, 1, figsize=(14, 12))

# ------------------------------------------------------------------
# MAIN MAP (Peninsula + Ceuta/Melilla)
# ------------------------------------------------------------------

# Base layer: All municipalities
gdf_mun[gdf_mun['Mun_Code_int'] < 53000].plot(
    ax=ax,
    color="lightgray",
    edgecolor="white",
    linewidth=0.1,
    zorder=1
)

# Provinces
gdf_prov.boundary.plot(
    ax=ax,
    linewidth=0.4,
    color="dimgray",
    zorder=2
)

# CCAA
gdf_ccaa.boundary.plot(
    ax=ax,
    linewidth=0.9,
    color="black",
    zorder=3
)

# LAYER 1: Historical mergers (BLUE)
if len(historical_mergers) > 0:
    historical_mergers.plot(
        ax=ax,
        color="#4287f5",  # Blue
        edgecolor="#1a5bbf",
        linewidth=0.8,
        alpha=0.8,
        zorder=4
    )

# LAYER 2: Shared land (RED)
shared_land.plot(
    ax=ax,
    color="#ff4444",  # Red
    edgecolor="#cc0000",
    linewidth=0.6,
    alpha=0.75,
    zorder=5
)

# LAYER 3: Special territories (YELLOW)
special_territories.plot(
    ax=ax,
    color="#ffdd00",  # Yellow
    edgecolor="#cc9900",
    linewidth=0.8,
    alpha=0.85,
    zorder=6
)

# ------------------------------------------------------------------
# Title
# ------------------------------------------------------------------
fig.suptitle(
    "TERRITORIAL ANOMALIES IN SPAIN",
    fontsize=18,
    fontweight="bold",
    y=0.95
)

ax.text(
    0.5, 0.93,
    "Non-standard territorial entities and recent municipal mergers",
    transform=fig.transFigure,
    ha="center",
    fontsize=12,
    style="italic"
)

# ------------------------------------------------------------------
# Legend (above scale bar)
# ------------------------------------------------------------------
legend_elements = [
    Line2D([0], [0], marker='s', color='w', label='Shared Land (53xxx)',
           markerfacecolor='#ff4444', markeredgecolor='#cc0000', markersize=10, linewidth=0),
    Line2D([0], [0], marker='s', color='w', label='Special Territories (54xxx)',
           markerfacecolor='#ffdd00', markeredgecolor='#cc9900', markersize=10, linewidth=0),
    Line2D([0], [0], marker='s', color='w', label='Historical Mergers',
           markerfacecolor='#4287f5', markeredgecolor='#1a5bbf', markersize=10, linewidth=0)
]

ax.legend(
    handles=legend_elements,
    loc='lower right',
    bbox_to_anchor=(0.99, 0.12),
    frameon=True,
    fancybox=True,
    shadow=True,
    fontsize=10
)

# ------------------------------------------------------------------
# North arrow (top-right)
# ------------------------------------------------------------------
arrow_coords = [
    (0.93, 0.90),  # top
    (0.91, 0.85),  # left
    (0.93, 0.87),  # center notch
    (0.95, 0.85)   # right
]

north_arrow = patches.Polygon(
    arrow_coords,
    closed=True,
    transform=ax.transAxes,
    facecolor="black",
    edgecolor="black",
    zorder=10
)

ax.add_patch(north_arrow)

ax.text(
    0.93, 0.91,
    "N",
    transform=ax.transAxes,
    ha="center",
    va="bottom",
    fontsize=11,
    fontweight="bold",
    color="black"
)

# ------------------------------------------------------------------
# Scale bar (200 km, bottom-right)
# ------------------------------------------------------------------
scale_text = "200 km"
x0, y0 = 0.87, 0.06
x1 = 0.99

ax.plot(
    [x0, x1], [y0, y0],
    transform=ax.transAxes,
    color="black",
    linewidth=2,
    zorder=10
)

ax.plot([x0, x0], [y0 - 0.01, y0 + 0.01],
        transform=ax.transAxes, color="black", linewidth=1)
ax.plot([x1, x1], [y0 - 0.01, y0 + 0.01],
        transform=ax.transAxes, color="black", linewidth=1)

ax.text(
    (x0 + x1) / 2,
    y0 - 0.02,
    scale_text,
    transform=ax.transAxes,
    ha="center",
    va="top",
    fontsize=9,
    color="black"
)

# ------------------------------------------------------------------
# Footer: CRS and author
# ------------------------------------------------------------------
fig.text(
    0.99, 0.01,
    f"CRS: {TARGET_CRS} (ETRS89) | © Juan Zotes Orcajo | Source: CNIG",
    ha="right",
    va="bottom",
    fontsize=8,
    color="dimgray"
)

# ------------------------------------------------------------------
# Peninsular extent
# ------------------------------------------------------------------
ax.set_xlim(-11.5, 5.5)
ax.set_ylim(34.8, 45.2)
ax.set_axis_off()

# ------------------------------------------------------------------
# CANARY ISLANDS INSET
# ------------------------------------------------------------------
CANARIAS_CODE = "05"

canarias_mun = gdf_mun[(gdf_mun["CCAA_Code"] == CANARIAS_CODE) & (gdf_mun['Mun_Code_int'] < 53000)]
canarias_prov = gdf_prov[gdf_prov["CCAA_Code"] == CANARIAS_CODE]
canarias_ccaa = gdf_ccaa[gdf_ccaa["CCAA_Code"] == CANARIAS_CODE]

ax_inset = inset_axes(
    ax,
    width="25%",
    height="25%",
    loc="lower left",
    borderpad=1.2
)

canarias_mun.plot(
    ax=ax_inset,
    color="lightgray",
    edgecolor="white",
    linewidth=0.1
)

canarias_prov.boundary.plot(
    ax=ax_inset,
    linewidth=0.4,
    color="dimgray"
)

canarias_ccaa.boundary.plot(
    ax=ax_inset,
    linewidth=0.9,
    color="black"
)

ax_inset.set_title("Canary Islands", fontsize=10, pad=4)
ax_inset.set_axis_off()

# Frame around inset
rect = patches.Rectangle(
    (0, 0), 1, 1,
    transform=ax_inset.transAxes,
    linewidth=1.0,
    edgecolor="black",
    facecolor="none",
    zorder=10
)
ax_inset.add_patch(rect)

plt.tight_layout()
plt.show()

## 5. Export map as PNG

In [ ]:
# Export map with DPI 369
output_file = OUTPUT_DIR / "territorial_anomalies_map.png"

fig.savefig(
    output_file,
    dpi=369,
    bbox_inches='tight',
    facecolor='white',
    edgecolor='none'
)

print("\n" + "="*70)
print("MAP EXPORTED")
print("="*70)
print(f"\n✓ File: {output_file}")
print(f"  Resolution: 369 DPI")
print(f"  Format: PNG")
print(f"  Size: {output_file.stat().st_size / 1024 / 1024:.2f} MB")

## 6. Summary statistics

In [ ]:
# Print summary
print("\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)

print(f"\nShared Land Entities (53xxx): {len(shared_land)}")
print(f"Special Territories (54xxx): {len(special_territories)}")
print(f"Historical Mergers: {len(historical_mergers)}")
print(f"\nTotal anomalies displayed: {len(shared_land) + len(special_territories) + len(historical_mergers)}")
print(f"Standard municipalities: {len(gdf_mun[gdf_mun['Mun_Code_int'] < 53000])}")